In [1]:
# install cassandra-sigv4
# %pip install cassandra-sigv4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 26.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


example code for connection:

In [5]:
# Set up a boto3 session and a Cassandra cluster (the Python way of interacting with AWS)
from cassandra.cluster import Cluster
from ssl import SSLContext, PROTOCOL_TLSv1_2, CERT_REQUIRED
from cassandra_sigv4.auth import SigV4AuthProvider
import boto3

# ssl setup
ssl_context = SSLContext(PROTOCOL_TLSv1_2)
ssl_context.load_verify_locations('/Users/caranugent/Desktop/Data Eng 300/sf-class2-root.crt')  # change your file path for locating the certificate
ssl_context.verify_mode = CERT_REQUIRED

# boto3 session setup
boto_session = boto3.Session(region_name="us-east-2")  # this AWS credentials is specific to `us-east-2` region

/var/folders/x2/m72jg43n4md074_09qnd1ml00000gn/T/ipykernel_54622/3644438619.py:8: DeprecationWarning: ssl.PROTOCOL_TLSv1_2 is deprecated
  ssl_context = SSLContext(PROTOCOL_TLSv1_2)


In [6]:
# authorization setup with SigV4
auth_provider = SigV4AuthProvider(boto_session)

In [7]:
#cluster setup 
cluster = Cluster(['cassandra.us-east-2.amazonaws.com'], 
                  ssl_context=ssl_context, 
                  auth_provider=auth_provider, 
                  port=9142)  # TLS only communicates on port 9142

Working with Cassandra (AWS Keyspace)

In [8]:
# establishing connection to Keyspace
session = cluster.connect()

In [9]:
# Insert any CQL queries between .connect() and .shutdown()

# For example, show all keyspaces created
r = session.execute('''
    SELECT * FROM system_schema.keyspaces;
    ''')
print(r.current_rows)

[Row(keyspace_name='system_schema', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='system_schema_mcs', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='system', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='system_multiregion_info', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='de300_axc4282', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='de300_demo', durable_writes=True, replication=OrderedMapSe

In [10]:
# For example, create a keyspace for HW2
r = session.execute('''
    CREATE KEYSPACE IF NOT EXISTS de300_demo 
    WITH replication = {'class': 'SingleRegionStrategy'};
    ''')
print(r.current_rows)

[]


***Exercises***
Let's first create a table within the keyspace 'de300-demo'. Note that when using Amazon Keyspaces, you must explicitly set an ExecutionProfile with LOCAL_QUORUM when creating your Cassandra Cluster in Python. Thus we set the credentials and reload the session

In [11]:
from cassandra.cluster import ExecutionProfile, EXEC_PROFILE_DEFAULT
from cassandra import ConsistencyLevel


# Define execution profile with LOCAL_QUORUM
execution_profile = ExecutionProfile(
    consistency_level=ConsistencyLevel.LOCAL_QUORUM
)

# Cluster setup with correct profile
cluster = Cluster(
    ['cassandra.us-east-2.amazonaws.com'],
    ssl_context=ssl_context,
    auth_provider=auth_provider,
    port=9142,
    execution_profiles={EXEC_PROFILE_DEFAULT: execution_profile}
)

# establishing connection to Keyspace
session = cluster.connect()
session.set_keyspace('de300_demo')  # Replace with your keyspace

create a new table named 'github'

In [12]:
session.execute("""
CREATE TABLE IF NOT EXISTS github (
    id UUID PRIMARY KEY,
    name TEXT,
    username TEXT
)
""")

Insert data to the table (please replace the value below with your name and GitHub username)

In [13]:
import uuid

session.execute("""
    INSERT INTO github (id, name, username)
    VALUES (%s, %s, %s)
""", (uuid.uuid4(), "Cara_Nugent", "caranugent"))

# Replace "Your_Name" with your name
# Replace "Your_GitHub_User_Name" with your real GitHub User

Export the table as a csv file

In [14]:
import csv

rows = session.execute("SELECT * FROM github")
with open("github.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["id", "name", "username"])
    for row in rows:
        writer.writerow([row.id, row.name, row.username])

Create a folder named "Lab4" in your GitHub repository, and upload the csv file there.